# Module Fitting

In [1]:
import geopandas as gpd
import matplotlib.pyplot as plt
import shapely
from shapely.geometry import Polygon
import numpy as np
from math import sin
import json

In [2]:
# gdf = gpd.read_file("sample_gdf.geojson")
gdf = gpd.read_file("building_assignment_output_filter25.geojson")
gdf.head()

,segment_id,building_id,address,image_id,category_id,geometry
0,90215,way/874972048,"C.M. Recto Street, 20 Phase 1, Bonanza, Fortun...",2986,12,"POLYGON ((121.12343 14.66166, 121.12343 14.661..."
1,112141,way/874972048,"C.M. Recto Street, 20 Phase 1, Bonanza, Fortun...",3707,5,"POLYGON ((121.12282 14.66165, 121.12281 14.661..."
2,53649,way/871081677,"128 Santan, Marikina, Metro Manila, Philippines",1775,13,"POLYGON ((121.12559 14.65968, 121.12559 14.659..."
3,90212,way/874972048,"C.M. Recto Street, 20 Phase 1, Bonanza, Fortun...",2986,5,"POLYGON ((121.12332 14.66190, 121.12332 14.661..."
4,19982,way/16827525,"Unit 2128, Riverbank Center, Riverbanks Avenue...",653,3,"POLYGON ((121.08306 14.63121, 121.08306 14.631..."


In [3]:
# roof geodataframe
roof_gdf = gdf.loc[(gdf['category_id'] != 1)].reset_index()
# ss geodataframe
ss_gdf = gdf.loc[(gdf['category_id'] == 1)].reset_index()

In [4]:
# shapely rotate func - Positive angles are counter-clockwise and negative are clockwise rotations

def rotate_polygons_south(roof_poly, category_id, ss_gdf):

    degrees_south = 180
    degrees_current = 0
    degrees_to_rotate = 0
    
    # WNW
    if category_id == 3: 
        degrees_current = 292.5
    
    # N
    elif category_id == 4:
        degrees_current = 360

    # NNW
    elif category_id == 5:
        degrees_current = 337.5

    # NW
    elif category_id == 6:
        degrees_current = 315

    # W
    elif category_id == 7:
        degrees_current = 270

    # WSW
    elif category_id == 8:
        degrees_current = 247.5

    # SW
    elif category_id == 9:
        degrees_current = 225
#     elif category_id == 9 or category_id == 2: # place flat roof panels on SW
#         degrees_current = 225

    # SSW
    elif category_id == 10:
        degrees_current = 202.5

    # S or FLAT
    elif category_id == 11 or category_id == 2: # no need to rotate flat since panels will be facing south by default
        degrees_current = 180
#     elif category_id == 11:
#         degrees_current = 180

    # SSE
    elif category_id == 12:
        degrees_current = 157.5

    # SE
    elif category_id == 13:
        degrees_current = 135

    # ESE
    elif category_id == 14:
        degrees_current = 112.5

    # E
    elif category_id == 15:
        degrees_current = 90

    # ENE
    elif category_id == 16:
        degrees_current = 67.5

    # NE
    elif category_id == 17:
        degrees_current = 45

    # NNE
    elif category_id == 18:
        degrees_current = 22.5
    
    else:
        degrees_current = 180

    degrees_to_rotate = degrees_current - degrees_south
    roof_centroid = roof_poly.centroid

    # rotate roof poly
    rotated_poly = shapely.affinity.rotate(roof_poly, degrees_to_rotate, roof_centroid)

    # rotate ss polys
    if len(ss_gdf) > 0:
        ss_gdf['geometry'] = ss_gdf['geometry'].apply(lambda x: shapely.affinity.rotate(x, degrees_to_rotate, roof_centroid))
   
    return rotated_poly, ss_gdf

In [5]:
def meters_to_degree(value):
    single_degree_in_meters = 111139
    meter_per_degree = 1.0 / single_degree_in_meters
    return meter_per_degree * value

def generate_grid(roof_poly,category_id, panel_length, panel_width):
    # poly = roof_poly
    poly = gpd.GeoSeries(roof_poly)
    xmin,ymin,xmax,ymax = poly.total_bounds

    # Panel specifications = 1.650m x 0.992m
    # length = meters_to_degree(1.65)
    # width = meters_to_degree(0.992)
    length = meters_to_degree(panel_length)
    width = meters_to_degree(panel_width)

    cols = list(np.arange(xmin, xmax, width))
    rows = list(np.arange(ymin, ymax+ 0.0000125, length))
   

    #SOH-CAH-TOA
    #get elevation height, using 30 deg elevation
    #sin(30) =(O)/panel_length
    #sin(30)(panel_length) = O
    height = sin(15) * panel_length
    height = abs(meters_to_degree(height))
 
    if category_id == 2:
        rows = list(np.arange(ymin, ymax+ 0.0000125, height))
        rows = [y for i, y in enumerate(rows, 1) if i % 3 == 0 and i != len(rows)]
       
 
    rows.reverse()
    polygons = []
    for x in cols:
        for y in rows:
            polygons.append( Polygon([(x,y), (x+width, y), (x+width, y-length), (x, y-length)]) )
    grid = gpd.GeoDataFrame({'geometry':polygons}, crs="EPSG:4326")
    return grid


def check_intersect_ss(roof_poly, panel_polys, ss_gdf):
    # extract list of ss polys from gdf
    ss_polys = list(ss_gdf['geometry'])

    polyA = roof_poly
    valid_panels = []

    for panel_poly in panel_polys:
        valid = True
        for ss_poly in ss_polys:
            if ss_poly.intersects(panel_poly) == True:
                valid = False
        if valid:
            valid_panels.append(panel_poly)

    return valid_panels
    

#check inside roof checks each polygon in the grid if it is inside the rooftop polygon
#function accepts three arguments
#rooftop poly = geodataframe containing the rooftop polygon / polygon
#grid = geodataframe containing the grid polygons
def check_inside_roof(roof_poly, grid):
    polyA = roof_poly
    inside = []
    for i in range(grid.shape[0]):
        polyB = grid.iloc[i]['geometry']
        if polyA.contains(polyB) == True:
            inside.append(polyB)
    return inside

def check_valid_ss(roof_poly, ss_gdf):
    valid = []
    for row in ss_gdf.itertuples():
        if roof_poly.intersects(row.geometry):
            valid.append(row.geometry)

    valid_gdf = gpd.GeoDataFrame({'geometry':valid}, crs="EPSG:4326")
    return valid_gdf


def module_fitting(roof_poly, category_id, ss_gdf, multiple_output=False):
    try:
        # get superstructures found within roof_poly
        ss_gdf = check_valid_ss(roof_poly, ss_gdf)

        # rotate roof and ss polys to South orientation
        roof_poly, ss_gdf = rotate_polygons_south(roof_poly, category_id, ss_gdf)

        # HORIZONTAL PANELS

        # generate grid
        grid_horizontal = generate_grid(roof_poly, category_id, 0.992, 1.65)

        # include panels that are inside the roof polygon
        panel_polys_horizontal = check_inside_roof(roof_poly, grid_horizontal)

        # exclude panels that intersect with superstructures
        if len(ss_gdf) > 0:
            panel_polys_horizontal = check_intersect_ss(roof_poly, panel_polys_horizontal, ss_gdf)


        # VERTICAL PANELS

        # generate grid
        grid_vertical = generate_grid(roof_poly,category_id, 1.65, 0.992)

        # include panels that are inside the roof polygon
        panel_polys_vertical = check_inside_roof(roof_poly, grid_vertical)

        # exclude panels that intersect with superstructures
        if len(ss_gdf) > 0:
            panel_polys_vertical = check_intersect_ss(roof_poly, panel_polys_vertical, ss_gdf)


        # return roof polygon, panel grid, valid panel polygons, superstructure polygons
        if len(panel_polys_vertical) > len(panel_polys_horizontal):
            if multiple_output:
                return roof_poly, grid_vertical, panel_polys_vertical, ss_gdf
            else:
                return len(panel_polys_vertical)
        else:
            if multiple_output:
                return roof_poly, grid_horizontal, panel_polys_horizontal, ss_gdf
            else:
                return len(panel_polys_horizontal)
    except:
        return -1

In [6]:
def plot_module_fitting(roof_poly, panel_polys, grid, ss_found_gdf):
    fig, ax = plt.subplots(figsize=(10,10))
    ax.set_aspect('equal')

    # plot roof polygon
    p = gpd.GeoSeries(roof_poly)
    p.plot(ax=ax, facecolor="grey", edgecolor='black')

    # plot grid
    grid.plot(ax=ax, facecolor="none")

    # plot superstructure on roof
    if len(ss_found_gdf) > 0:
        ss_found_gdf.plot(ax=ax, facecolor="red", edgecolor='black')

    # plot all valid panel polygons
    for i in range(len(panel_polys)):
        poly_panel = panel_polys[i]
        p = gpd.GeoSeries(poly_panel)
        p.plot(ax=ax, facecolor="green", edgecolor='black')

    plt.show()

    print("Number of panels:", len(panel_polys))

In [7]:
# calculate area of valid panels (m^2)
def calculate_area(num_panels):
    # Panel specifications = 1.650m x 0.992m
    length = 1.65
    width = 0.992
    return length * width * num_panels

In [10]:
# Fixes possible issues with intersections
roof_gdf['geometry'] = roof_gdf.buffer(0)
ss_gdf['geometry'] = ss_gdf.buffer(0)

In [11]:
from shapely import affinity

def scale_polygon(polygon, scale):
    if scale == 'up':
        return affinity.scale(polygon, xfact=1.2, yfact=1.2,origin='centroid')
    else:
        return affinity.scale(polygon, xfact=0.95, yfact=0.95,origin='centroid')

In [12]:
# scale-up roof polygons and scale-down ss polygons to add buffer
roof_gdf['geometry'] = roof_gdf.apply(lambda x: scale_polygon(x['geometry'], scale='down'), axis=1)
ss_gdf['geometry'] = ss_gdf.apply(lambda x: scale_polygon(x['geometry'], scale='up'), axis=1)

# map module_fitting function to roof geodataframe
valid_roof_gdf = roof_gdf.copy()
valid_roof_gdf['num_panels'] = roof_gdf.apply(lambda x: module_fitting(x['geometry'], x['category_id'], ss_gdf), axis=1)

/home/jupyter-sinagpala/.conda/envs/pipeline/lib/python3.9/site-packages/pandas/core/dtypes/inference.py:383: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  iter(obj)  # Can iterate over it.
/home/jupyter-sinagpala/.conda/envs/pipeline/lib/python3.9/site-packages/pandas/core/dtypes/inference.py:384: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  len(obj)  # Has a length associated with it.
/home/jupyter-sinagpala/.conda/envs/pipeline/lib/python3.9/site-packages/pandas/core/dtypes/cast.py:1990: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number o

In [13]:
valid_roof_gdf = valid_roof_gdf[valid_roof_gdf['num_panels'] != -1]

In [14]:
# map calculate_area function to roof geodataframe
valid_roof_gdf['panel_area'] = valid_roof_gdf.apply(lambda x: calculate_area(x['num_panels']),axis=1)

In [15]:
valid_roof_gdf

,index,segment_id,building_id,address,image_id,category_id,geometry,num_panels,panel_area
0,0,90215,way/874972048,"C.M. Recto Street, 20 Phase 1, Bonanza, Fortun...",2986,12,"MULTIPOLYGON (((121.12337 14.66163, 121.12339 ...",1160,1898.6880
1,1,112141,way/874972048,"C.M. Recto Street, 20 Phase 1, Bonanza, Fortun...",3707,5,"POLYGON ((121.12283 14.66167, 121.12283 14.661...",888,1453.4784
2,2,53649,way/871081677,"128 Santan, Marikina, Metro Manila, Philippines",1775,13,"POLYGON ((121.12561 14.65969, 121.12560 14.659...",740,1211.2320
3,3,90212,way/874972048,"C.M. Recto Street, 20 Phase 1, Bonanza, Fortun...",2986,5,"POLYGON ((121.12333 14.66191, 121.12333 14.661...",734,1201.4112
4,4,19982,way/16827525,"Unit 2128, Riverbank Center, Riverbanks Avenue...",653,3,"POLYGON ((121.08306 14.63122, 121.08306 14.631...",710,1162.1280
...,...,...,...,...,...,...,...,...,...
102146,111496,64936,way/120433106,"5 Shorthorn St, Marikina, 1800 Metro Manila, P...",2128,8,"POLYGON ((121.12277 14.64347, 121.12277 14.643...",20,32.7360
102147,111497,90342,way/160697218,"16 St Bernadette, Marikina, 1803 Metro Manila,...",2991,2,"POLYGON ((121.08778 14.63301, 121.08778 14.633...",6,9.8208
102148,111498,23247,way/119756773,"7 Topaz, Marikina, 1800 Rizal, Philippines",764,11,"POLYGON ((121.12089 14.63662, 121.12089 14.636...",6,9.8208
102149,111499,83166,way/160697167,"1800 Liverpool, Marikina, 1800 Metro Manila, P...",2746,11,"POLYGON ((121.09053 14.63163, 121.09053 14.631...",9,14.7312


In [16]:
# valid_roof_gdf.to_file(filename='roof_gdf_module_fitting_flatsouth.geojson', driver='GeoJSON')
valid_roof_gdf.to_file(filename='roof_gdf_module_fitting_flatsw.geojson', driver='GeoJSON')